In [1]:
import sys
sys.path.append("/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/")
from pathlib import Path
from itertools import product
from tqdm import tqdm
import time
import os
import torch
from nn_models import train_model
import copy
import numpy as np
from sklearn.model_selection import train_test_split
import h5py
import traceback

## Data preprocessing

### Split the data first

In [ ]:
data_dir = "/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Lab_data_finetune/Walking_1/split/s23_2" 
train_dir = "/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Lab_data_finetune/Walking_1/split/s23_5"
test_dir = "/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Lab_data_finetune/Walking_1/split/s23_6"

split_ratio = 0.5  # 80% train, 20% test

def save_split_data(data, train_path, test_path):
    os.makedirs(os.path.dirname(train_path), exist_ok=True)
    os.makedirs(os.path.dirname(test_path), exist_ok=True)

    train_data, test_data = train_test_split(data, train_size=split_ratio, shuffle=False)
    np.save(train_path, train_data)
    np.save(test_path, test_data)

for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".npy"):
            full_path = os.path.join(root, file)
            rel_path = os.path.relpath(full_path, data_dir)
            
            
            train_path = os.path.join(train_dir, rel_path)
            test_path = os.path.join(test_dir, rel_path)

        
            data = np.load(full_path)
            save_split_data(data, train_path, test_path)


### Convert into h5

In [3]:
# Set the base data folder and output file name
base_dir = '/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Lab_data_finetune/Walking'
output_path = 'Walking.h5'

with h5py.File(output_path, 'w') as h5file:
    for subject in os.listdir(base_dir):
        subject_path = os.path.join(base_dir, subject)
        if not os.path.isdir(subject_path):
            continue  # Skip non-directory files

        for body_part in os.listdir(subject_path):
            part_path = os.path.join(subject_path, body_part)
            if not os.path.isdir(part_path):
                continue

            for file in os.listdir(part_path):
                if file.endswith('.npy'):
                    file_path = os.path.join(part_path, file)
                    data = np.load(file_path)

                    # Remove .npy extension, e.g., 'acc.npy' -> 'acc'
                    dataset_name = os.path.splitext(file)[0]

                    # Build the group path in the HDF5 file, e.g., s22_1/LeftFoot/acc
                    group_path = f"{subject}/{body_part}/{dataset_name}"

                    # Save the dataset
                    h5file.create_dataset(group_path, data=data)

print(f"✅ Successfully saved HDF5 file to: {output_path}")


✅ Successfully saved HDF5 file to: Walking.h5


In [6]:
RENAME_MAP = {
    'LeftFoot': 'lfoot',
    'LeftShank': 'lshank',
    'LeftKneeAngle': 'lknee',
    'LeftThigh': 'lthigh',
    'LeftHipAngle': 'lhip',
    'LeftAnkleAngle': 'lankle',
    'RightFoot': 'rfoot',
    'RightShank': 'rshank',
    'RightKneeAngle': 'rknee',
    'RightThigh': 'rthigh',
    'RightHipAngle': 'rhip',
    'RightAnkleAngle': 'rankle',
    'Pelvis':'pelvis'
}


def rename_folders(base_path):
    for root, dirs, files in os.walk(base_path):
        for name in dirs:
            if name in RENAME_MAP:
                old_path = os.path.join(root, name)
                new_path = os.path.join(root, RENAME_MAP[name])
                print(f"🔄 Renaming {old_path} → {new_path}")
                os.rename(old_path, new_path)

if __name__ == '__main__':
    base_dir = '/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Lab_data_finetune/Walking_1/split/s23_4'  # ← 換成你 Running 資料夾的絕對路徑
    rename_folders(base_dir)
    print("✅ Rename done.")


🔄 Renaming /Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Lab_data_finetune/Walking_1/split/s23_4/LeftKneeAngle → /Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Lab_data_finetune/Walking_1/split/s23_4/lknee
🔄 Renaming /Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Lab_data_finetune/Walking_1/split/s23_4/RightKneeAngle → /Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Lab_data_finetune/Walking_1/split/s23_4/rknee
🔄 Renaming /Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Lab_data_finetune/Walking_1/split/s23_4/LeftShank → /Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Lab_data_finetune/Walking_

In [3]:
with h5py.File('/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Lab_data_finetune/Walking.h5', 'r') as f:
    def print_tree(name, obj):
        print(name)
    f.visititems(print_tree)


s1
s1/lankle
s1/lankle/angle
s1/lfoot
s1/lfoot/acc
s1/lfoot/gyr
s1/lhip
s1/lhip/angle
s1/lknee
s1/lknee/angle
s1/lshank
s1/lshank/acc
s1/lshank/gyr
s1/lthigh
s1/lthigh/acc
s1/lthigh/gyr
s1/pelvis
s1/pelvis/acc
s1/pelvis/gyr
s1/rankle
s1/rankle/angle
s1/rfoot
s1/rfoot/acc
s1/rfoot/gyr
s1/rhip
s1/rhip/angle
s1/rknee
s1/rknee/angle
s1/rshank
s1/rshank/acc
s1/rshank/gyr
s1/rthigh
s1/rthigh/acc
s1/rthigh/gyr
s2
s2/lankle
s2/lankle/angle
s2/lfoot
s2/lfoot/acc
s2/lfoot/gyr
s2/lhip
s2/lhip/angle
s2/lknee
s2/lknee/angle
s2/lshank
s2/lshank/acc
s2/lshank/gyr
s2/lthigh
s2/lthigh/acc
s2/lthigh/gyr
s2/pelvis
s2/pelvis/acc
s2/pelvis/gyr
s2/rankle
s2/rankle/angle
s2/rfoot
s2/rfoot/acc
s2/rfoot/gyr
s2/rhip
s2/rhip/angle
s2/rknee
s2/rknee/angle
s2/rshank
s2/rshank/acc
s2/rshank/gyr
s2/rthigh
s2/rthigh/acc
s2/rthigh/gyr
s3
s3/lankle
s3/lankle/angle
s3/lfoot
s3/lfoot/acc
s3/lfoot/gyr
s3/lhip
s3/lhip/angle
s3/lknee
s3/lknee/angle
s3/lshank
s3/lshank/acc
s3/lshank/gyr
s3/lthigh
s3/lthigh/acc
s3/lthigh/gyr


## Finetune

In [ ]:
def finetune_transformer(
    joint='Knee',
    activity='Walking',
    base_path='/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Lab_data_finetune',
    result_folder='/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/result/finetuned_results',
    base_weight_path='/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/model/tranformer/angle_model',  # 新增的權重路徑根目錄
    num_iter=1000,
    batch_size=2,
    seq_len=100,
    lr=0.001,
    lr_decay=0.98
):

    #joint = joint.lower()
    #activity = activity.lower()

    assert activity in ['Walking', 'Running']
    assert joint in ['Hip', 'Knee', 'Ankle']

    data_path = os.path.join(base_path, f'{activity}.h5')
    result_path = os.path.join(result_folder, activity, joint)
    os.makedirs(result_path, exist_ok=True)

    pretrained_model_path = os.path.join(base_weight_path, activity, joint)
    if not os.path.exists(os.path.join(pretrained_model_path, 'model.pt')):
        raise FileNotFoundError(f"[X] model.pt not found in: {pretrained_model_path}")

    # General spec
    general_spec = {  
        'model_type': 'transformer',
        'data_path': data_path,
        'result_path': result_path + '/',
        'inp': ['pelvis', 'thigh'] if joint == 'Hip'
        else ['thigh', 'shank'] if joint == 'Knee'
        else ['shank', 'foot'],
        'outp': [joint.lower()],
        'log_metrics': ['loss', 'lr', 'rmse'],
        'log_freq': 100,
        'check_freq': 500,
        'optim': 'Adam',
        'lr': lr,
        'loss': 'MSELoss',
        'scheduler': 'ExponentialLR',
        'lr_decay': lr_decay,
        'prog': True,
        'aug': False,
        'x_noise': 0.1,
        'y_noise': 0,
        'seq_len': [seq_len],
        'num_iter': [num_iter],
        'batch_size': [batch_size],
        'pretrained': pretrained_model_path }

    # Model config for Transformer
    model_spec = {
        'inp_size': [8 * len(general_spec['inp'])],
        'outp_size': [3 * len(general_spec['outp'])],
        'd_model': 96,
        'num_heads': 6,
        'num_layers': 10,
        'max_seq_len': 200,
        'prediction': 'angle'
    }

    general_spec['name'] = f"finetuned_{joint}_{activity}_transformer_iter{num_iter}_lr{lr}"

    train_model.main(copy.deepcopy(general_spec), copy.deepcopy(model_spec), show_bar=True)
    torch.cuda.empty_cache()

    print(f"[✓] Fine-tuning finished: {activity}-{joint}")
    print(f"Saved to: {general_spec['result_path']}")

In [5]:
def finetune_all_models(
    activities=('Walking',),
    joints=('Hip', 'Knee', 'Ankle'),
    base_path='/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Lab_data_finetune',
    result_folder='/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/result/finetuned_results',
    base_weight_path='/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/model/tranformer/angle_model',
    num_iter=1000,
    batch_size=2,
    seq_len=100,
    lr=0.0001,
    lr_decay=0.98
):

    combinations = list(product(activities, joints))
    print(f"\n🔁 Fine-tuning {len(combinations)} combinations...\n")

    for activity, joint in tqdm(combinations):
        try:
            print(f"\n🚀 Fine-tuning: {activity.upper()} - {joint.upper()}")
            finetune_transformer(
                joint=joint,
                activity=activity,
                base_path=base_path,
                base_weight_path=base_weight_path,
                result_folder=result_folder,
                num_iter=num_iter,
                batch_size=batch_size,
                seq_len=seq_len,
                lr=lr,
                lr_decay=lr_decay
            )
        except Exception as e:
            print(f"[X] Failed: {activity}-{joint} -> {e}")
            traceback.print_exc()
        time.sleep(1)  

    print("\n✅ All fine-tuning complete!")

if __name__ == '__main__':
    finetune_all_models()


🔁 Fine-tuning 3 combinations...



  0%|          | 0/3 [00:00<?, ?it/s]


🚀 Fine-tuning: WALKING - HIP
Using device: cpu

transformer(
  (input_linear): Linear(in_features=16, out_features=96, bias=True)
  (layers): ModuleList(
    (0-9): 10 x TransformerEncoderLayerRoPE(
      (self_attn): MultiHeadAttentionRoPE(
        (qkv_proj): Linear(in_features=96, out_features=288, bias=True)
        (out_proj): Linear(in_features=96, out_features=96, bias=True)
      )
      (linear1): Linear(in_features=96, out_features=256, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=256, out_features=96, bias=True)
      (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
  )
  (out_linear): Linear(in_features=96, out_features=3, bias=True)
)
Total number of trainable parameters: 873283
path: /Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Te

  0%|          | 0/3 [00:24<?, ?it/s]


KeyboardInterrupt: 